# Карташов Никита Андреевич, гр. 0373

## Теория принятия решений. Вариант 140


### Задача 2

Технология краудсорсинга позволяет привлекать широкий круг людей к задачам сбора и обработки информации. Заказчики могут размещать на платформе краудсорсинга задания, назначая за них определенное вознаграждение, а пользователи платформы (исполнители) выполняют эти задания и получают назначенное вознаграждение. Примерами таких платформ являются Amazon Mechanical Turk, Яндекс.Толока. Одной из разновидностей краудсорсинга является пространственный краудсорсинг, при котором задания имеют пространственную привязку (например, с помощью системы пространственного краудсорсинга можно сделать фотографию определенной географической локации в определенный момент).

В данной задаче рассматривается один из центральных компонентов платформы пространственного краудсорсинга, осуществляющий распределение заданий среди исполнителей.

Задано определенное множество заданий (табл. 1). Каждое задание имеет пространственную привязку. Одной из функций подобной привязки являтся идентификация исполнителей, находящихся рядом с заданием. Для простоты будем полагать, что задания и исполнители находятся в городе Гипподамовой системы, что позволяет использовать локальную систему координат и оценивать расстояние с помощью манхеттенской метрики. Задания считаются достаточно простыми, чтобы любое из них могло быть выполнено любым участником.

В заданный момент в системе активно определенное множество участников (табл. 2). Для каждого участника задано текущее положение в локальной системе координат, а также текущий рейтинг (в диапазоне от 0 до 1 - чем больше, тем более надежным является исполнитель).

Вознаграждение исполнителя определяется его рейтингом. В частности, задано два граничных уровня рейтинга $Th_{M}=0.3$ (средний) и $Th_{H}=0.7$ (высокий). Участники с высоким рейтингом ($r_{i} \geq Th_{H}$) получают вознаграждение 110 денежных единиц (ДЕ) за каждое задание, участники со средним рейтингом ($Th_{M} \leq r_{i} \lt Th_{H}$) получают вознаграждение 60 ДЕ.
Участники с рейтингом ниже $Th_{M}$ считаются заблокированными и к выполнению заданий не привлекаются.

Бюджет заказчика на выполнение заданного набора заданий составляет 320 ДЕ.

Требуется:

1. Считая качество выполнения задания численно равным рейтингу исполнителя, найти распределение (всех) заданий по исполнителям, максимизирующее суммарное качество выполнения заданий.

2. Модифицировать целевую функцию с учетом расстояния между заданием и исполнителем (считая, что ожидаемое качество выполнения задания убывает как логарифм от расстояния) и найти распределение заданий с учетом расстояния.


_Таблица 1: Задания_
| Идентификатор | x | y |
|---------------|----|----|
| 1 | 9 | 11 |
| 2 | 10 | 4 |
| 3 | 9 | 8 |
| 4 | 1 | 12 |


_Таблица 2: Исполнители_
| Идентификатор | x | y | Рейтинг |
|---------------|----|----|---------|
| 1 | 17 | 16 | 0.93 |
| 2 | 10 | 10 | 0.35 |
| 3 | 7 | 2 | 0.78 |
| 4 | 15 | 18 | 0.75 |
| 5 | 6 | 14 | 0.83 |
| 6 | 11 | 7 | 0.23 |
| 7 | 14 | 15 | 0.73 |
| 8 | 18 | 0 | 0.32 |
| 9 | 17 | 9 | 0.91 |
| 10 | 15 | 14 | 0.26 |


### Пункт 1.

#### 1.1 Импортируем необходимые для решения модули:


In [533]:
import math
import numpy as np
from cvxopt import matrix, glpk, solvers
from cvxopt.modeling import variable, op, blas, dot


#### 1.2 Введём начальные данные:


In [534]:
tasks = [[9, 11], [10, 4], [9, 8], [1, 12]]
budget = 340.0
users = [
    {"coords": [17, 16], "rating": 0.93},
    {"coords": [10, 10], "rating": 0.35},
    {"coords": [7, 2], "rating": 0.78},
    {"coords": [15, 18], "rating": 0.75},
    {"coords": [6, 14], "rating": 0.83},
    {"coords": [11, 7], "rating": 0.23},
    {"coords": [14, 15], "rating": 0.73},
    {"coords": [18, 0], "rating": 0.32},
    {"coords": [17, 9], "rating": 0.91},
    {"coords": [15, 14], "rating": 0.26},
]

#### 1.3 Рассчитаем, какие пользователи могут быть допущены к выполнению задания, а какие нет. Также рассчитаем стоимость их услуг:


In [535]:
activeUsers = matrix([users[i]["rating"] for i in range(len(users))], tc="d").T
availableUsers = matrix(
    [[activeUsers[i] if activeUsers[i] >= 0.3 else 0]
        for i in range(len(activeUsers))],
    tc="d",
)
unavailableUsers = activeUsers - availableUsers
cost = matrix(
    [
        (110 if activeUsers[i] >= 0.7 else 60) if (
            activeUsers[i] >= 0.3) else (0)
        for i in range(len(activeUsers))
    ],
    tc="d",
)

#### 1.4 Составим целевую функцию и набор ограничений:


In [536]:
targetFunction = matrix(availableUsers, tc="d")
inequalityLeft = matrix([cost], tc="d").T
inequalityRight = matrix([budget], tc="d")
equalityLeft = matrix(
    [matrix([[1] for i in range(len(activeUsers))]), unavailableUsers], tc="d"
)
equalityRight = matrix([len(tasks), 0], tc="d")

#### 1.5 С помощью солвера "glpk" решим задачу по максимизации целевой функции с учётом всех ограничений. Принимаем во внимание, что получаемые нами переменные $x_{i}$ должны представлять собой булевый тип:


In [537]:
(status, x) = glpk.ilp(
    c=-targetFunction,
    G=inequalityLeft,
    h=inequalityRight,
    A=equalityLeft,
    b=equalityRight,
    B=set(range(len(activeUsers))),
)

#### 1.6 Интерпретируем результат:


In [538]:
print(f"Статус: {status}")
print(
    f"Вектор получившихся переменных X:\n{x}"
    if (status == "optimal")
    else "Задача не имеет оптимального решения."
)
# print(matrix([[x],[targetFunction.T], [cost]]))


Статус: optimal
Вектор получившихся переменных X:
[ 1.00e+00]
[ 1.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 1.00e+00]
[ 1.00e+00]
[ 0.00e+00]



### Пункт 2.

#### 2.1 :


In [539]:
volume = len(tasks) * len(users)
oneManToTask = np.array([])
for i in range(0, volume, len(users)):
    temp = np.concatenate(
        (np.zeros(i), np.ones(len(users)), np.zeros(volume - len(users) - i))
    )
    if i == 0:
        oneManToTask = np.copy(temp)
    else:
        oneManToTask = np.vstack((oneManToTask, temp))

In [540]:
oneTaskMax = np.array([])
for i in range(len(users)):
    temp = np.concatenate(
        (np.zeros(i), np.ones(1), np.zeros(len(users) - i - 1)))
    temp = np.tile(temp, len(tasks))
    if i == 0:
        oneTaskMax = np.copy(temp)
    else:
        oneTaskMax = np.vstack((oneTaskMax, temp))

In [541]:
targetFunction = matrix([availableUsers.T for i in range(len(tasks))], tc="d").T
inequalityLeft = matrix([[cost for i in range(len(tasks))], [matrix(oneTaskMax).T]], tc="d").T
inequalityRight = matrix([budget, matrix(np.ones(len(users)))], tc="d")
print(inequalityRight)
equalityLeft = matrix(
    [
        [matrix([[1] for i in range(len(tasks) * len(activeUsers))]).T],
        [matrix([unavailableUsers.T for i in range(len(tasks))])],
        [matrix(oneManToTask).T],
    ],
    tc="d",
).T
equalityRight = matrix([len(tasks), 0, matrix(np.ones(len(tasks)))], tc="d")

[ 3.40e+02]
[ 1.00e+00]
[ 1.00e+00]
[ 1.00e+00]
[ 1.00e+00]
[ 1.00e+00]
[ 1.00e+00]
[ 1.00e+00]
[ 1.00e+00]
[ 1.00e+00]
[ 1.00e+00]



In [542]:
(status2, x2) = glpk.ilp(
    c=-targetFunction,
    G=inequalityLeft,
    h=inequalityRight,
    A=equalityLeft,
    b=equalityRight,
    B=set(range(volume)),
)
print(f"Статус: {status2}")
print("Распределение по заданиям:")
for i in range(len(tasks)):
    resultSlice = x2[i*len(users) : i*len(users) + len(users)]
    for j in range(len(users)):
        if(resultSlice.T[j] > 0):
            print(f"\nЗадание {1+i}: Участник {1+j}")
            print(f"Рейтинг участника: {users[j]['rating']}")
            print(f"Оплата: {cost[j]}")
            break
    # print(
    #     f"Распределение на задание {1+i}:\n{resultSlice}")
# и коэффициенты расстояний

Статус: optimal
Распределение по заданиям:

Задание 1: Участник 2
Рейтинг участника: 0.35
Оплата: 60.0

Задание 2: Участник 8
Рейтинг участника: 0.32
Оплата: 60.0

Задание 3: Участник 1
Рейтинг участника: 0.93
Оплата: 110.0

Задание 4: Участник 9
Рейтинг участника: 0.91
Оплата: 110.0
